In [54]:
import pandas as pd
import numpy as np

from statsmodels.tsa.seasonal import seasonal_decompose
from scipy.stats import boxcox

In [6]:
potato = pd.read_csv('potato_data.csv')

data_prices = potato[['Modal Price (Rs./Quintal)', 'Variety', 'Reported Date']]

data = data.copy()

data.loc[:, 'Modal Price (Rs./Quintal)'] = data['Modal Price (Rs./Quintal)'].astype(float)
data['Reported Date'] = pd.to_datetime(data['Reported Date'], format = '%d-%b-%y')

data.head()

,Modal Price (Rs./Quintal),Variety,Reported Date
0,900,(Red Nanital),2019-11-01
1,970,(Red Nanital),2019-10-19
2,500,(Red Nanital),2022-11-18
3,300,(Red Nanital),2022-03-08
4,600,(Red Nanital),2022-02-15


In [8]:
data.isnull().sum()

Modal Price (Rs./Quintal)    0
Variety                      0
Reported Date                0
dtype: int64

In [19]:
data.shape

(8052, 3)

In [21]:
sorted_data = data.sort_values(by = ['Reported Date', 'Variety'])
filtered_data = sorted_data[sorted_data['Reported Date'] != '2002-03-21']
filtered_data.head()

,Modal Price (Rs./Quintal),Variety,Reported Date
376,525,Desi,2002-11-01
4250,575,Desi,2002-11-02
1551,625,Desi,2002-11-06
743,575,Desi,2002-11-07
2510,625,Desi,2002-11-08


In [22]:
filtered_data.shape

(8051, 3)

In [23]:
data_indexed = filtered_data.copy()
data_indexed.set_index('Reported Date', inplace=True)
data_indexed.head()

,Modal Price (Rs./Quintal),Variety
Reported Date,,
2002-11-01,525,Desi
2002-11-02,575,Desi
2002-11-06,625,Desi
2002-11-07,575,Desi
2002-11-08,625,Desi


In [29]:
## one-hot encoding for variety
one_hot = pd.get_dummies(data_indexed['Variety'],prefix = 'Variety')
data_combined = data_indexed.drop('Variety', axis=1).join(one_hot)

aggregation = {col: 'first' for col in data_combined.columns if not col.startswith('Variety_')}
aggregation.update({col : 'sum' for col in data_combined.columns if col.startswith('Variety_')})

data_grouped = data_combined.groupby(data_combined.index).agg(aggregation)

data_grouped.head()

,Modal Price (Rs./Quintal),Variety_(Red Nanital),Variety_Chips,Variety_Desi,Variety_Jyoti,Variety_Local,Variety_Other,Variety_Potato,Variety_Red
Reported Date,,,,,,,,,
2002-11-01,525,0,0,1,0,0,0,0,0
2002-11-02,575,0,0,1,0,0,0,0,0
2002-11-06,625,0,0,1,0,0,0,0,0
2002-11-07,575,0,0,1,0,0,0,0,0
2002-11-08,625,0,0,1,0,0,0,0,0


In [30]:
data_grouped.shape

(5237, 9)

In [35]:
all_dates = pd.date_range(start = data_grouped.index.min(), end = data_grouped.index.max())
data_reindexed = data_grouped.reindex(all_dates)
data_reindexed = data_reindexed.reset_index().rename(columns = {'index' : 'Date'})
data_reindexed = data_reindexed.set_index('Date')
data_reindexed.head()

,Modal Price (Rs./Quintal),Variety_(Red Nanital),Variety_Chips,Variety_Desi,Variety_Jyoti,Variety_Local,Variety_Other,Variety_Potato,Variety_Red
Date,,,,,,,,,
2002-11-01,525.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2002-11-02,575.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2002-11-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002-11-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002-11-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
for col in data_reindexed.columns:
    data_reindexed[col] = data_reindexed[col].astype(float)

data_reindexed = data_reindexed.ffill().bfill()
df = data_reindexed.copy()

In [45]:
df.shape

(7922, 9)

In [51]:
decomposition = seasonal_decompose(df['Modal Price (Rs./Quintal)'], model='additive', period=30)
df['trend'] = decomposition.trend
df['seasonal'] = decomposition.seasonal
df['residual'] = decomposition.resid

df2 = df.copy()
df2.dropna(inplace=True)

df2.head()

,Modal Price (Rs./Quintal),Variety_(Red Nanital),Variety_Chips,Variety_Desi,Variety_Jyoti,Variety_Local,Variety_Other,Variety_Potato,Variety_Red,trend,seasonal,residual
Date,,,,,,,,,,,,
2002-11-16,600.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,591.000000,1.476728,7.523272
2002-11-17,600.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,593.166667,0.567195,6.266138
2002-11-18,600.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,594.500000,14.315802,-8.815802
2002-11-19,525.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,595.833333,-0.714997,-70.118337
2002-11-20,625.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,597.166667,14.212380,13.620954


In [61]:
df2['rolling_mean'] = df2['Modal Price (Rs./Quintal)'].rolling(window=15).mean()
df2['rolling_std'] = df2['Modal Price (Rs./Quintal)'].rolling(window=15).std()
df2['rolling_sum'] = df2['Modal Price (Rs./Quintal)'].rolling(window=15).sum()

df2 = df2.bfill()
df2.shape

(7890, 16)

In [62]:
# Check for non-positive values
non_positive_values = df2[df2['Modal Price (Rs./Quintal)'] <= 0]

# If non-positive values are found, handle them
if not non_positive_values.empty:
    # Option 1: Remove non-positive values
    df2 = df2[df2['Modal Price (Rs./Quintal)'] > 0]

In [63]:
df2.shape

(7890, 16)

In [64]:
df2['modal_boxcox'], lam = boxcox(df2['Modal Price (Rs./Quintal)'])

In [65]:
df2.shape

(7890, 16)

In [66]:
df2.to_csv('data_for_lstm_model3.csv')